ComfyUI server

In [2]:
# --- CELL 1: UNIVERSAL INSTALLER & SERVER START ---
import os
import time
import subprocess
from google.colab import drive, userdata

# 1. Mount Drive
drive.mount('/content/drive')

# 2. Install ComfyUI
print("📦 Installing ComfyUI...")
%cd /content
if not os.path.exists("ComfyUI"):
    !git clone https://github.com/comfyanonymous/ComfyUI.git
    %cd /content/ComfyUI
    !pip install -r requirements.txt
    !pip install websocket-client
else:
    %cd /content/ComfyUI

# 3. Install ComfyUI Manager
print("📦 Installing ComfyUI Manager...")
%cd /content/ComfyUI/custom_nodes
if not os.path.exists("ComfyUI-Manager"):
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git
    %cd ComfyUI-Manager
    !pip install -r requirements.txt

# 4. Install Video Helper Suite (VHS) - For Video
print("📦 Installing Video Helper Suite...")
%cd /content/ComfyUI/custom_nodes
if not os.path.exists("ComfyUI-VideoHelperSuite"):
    !git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
    %cd ComfyUI-VideoHelperSuite
    !pip install -r requirements.txt

# 5. Download Models (Video & Image)
print("⬇️ Downloading Upscale Models...")
%cd /content/ComfyUI/models/upscale_models
# For Video (Fast)
!wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth
# For Video (High Quality)
!wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
# For Images (Best Photo Quality)
!wget -nc https://huggingface.co/lokCX/4x-Ultrasharp/resolve/main/4x-UltraSharp.pth

# 6. Ngrok Setup
print("🔑 Setting up Ngrok...")
!pip install pyngrok
from pyngrok import ngrok, conf
ngrok.kill()

try:
    token = userdata.get('ngrok')
    if token:
        conf.get_default().auth_token = token
        print("✅ Ngrok token loaded.")
    else:
        print("⚠️ Secret 'ngrok' not found. Add it to Colab secrets for a GUI link.")
except:
    pass

# 7. Start Server (Optimized)
%cd /content/ComfyUI
!pkill -f main.py
if os.path.exists("comfyui.log"): os.remove("comfyui.log")

print("🚀 Starting ComfyUI (FP16 Mode)...")
!nohup python main.py --listen --port 8188 --force-fp16 --fast > comfyui.log 2>&1 &
time.sleep(15)

# 8. GUI Link
try:
    public_url = ngrok.connect(8188).public_url
    print(f"\n✅ GUI LINK: {public_url}\n")
except:
    print("❌ No Ngrok token found (GUI link skipped). Automation will still work.")

MessageError: Error: credential propagation was unsuccessful

Google Drive Watcher

In [ ]:
# --- CELL 2: UNIVERSAL WATCHER (IMAGES & VIDEO) ---
import os
import time
import json
import urllib.request
import uuid
import shutil
from google.colab import drive

# --- CONFIGURATION ---
BASE_DIR = '/content/drive/MyDrive/image-and-video-upscale'
WATCH_FOLDER = os.path.join(BASE_DIR, 'input_files')
OUTPUT_FOLDER = os.path.join(BASE_DIR, 'output_files')
PROCESSED_FOLDER = os.path.join(BASE_DIR, 'processed_files')

# Workflow Files
VIDEO_JSON = os.path.join(BASE_DIR, 'video-upscale-workflow-api.json')
IMAGE_JSON = os.path.join(BASE_DIR, 'image-upscale-workflow-api.json')

SERVER_ADDRESS = "127.0.0.1:8188"
COMFY_INPUT = '/content/ComfyUI/input'
COMFY_OUTPUT = '/content/ComfyUI/output'

# Ensure folders exist
for f in [WATCH_FOLDER, OUTPUT_FOLDER, PROCESSED_FOLDER]:
    os.makedirs(f, exist_ok=True)

# --- UTILS ---
def queue_prompt(workflow_data):
    p = {"prompt": workflow_data, "client_id": str(uuid.uuid4())}
    data = json.dumps(p).encode('utf-8')
    req = urllib.request.Request(f"http://{SERVER_ADDRESS}/prompt", data=data)
    return json.loads(urllib.request.urlopen(req).read())

def get_history(prompt_id):
    try:
        with urllib.request.urlopen(f"http://{SERVER_ADDRESS}/history/{prompt_id}") as response:
            return json.loads(response.read())
    except:
        return {}

def handle_video(file_path, filename):
    print("🎬 Detected VIDEO. Loading Video Workflow...")
    if not os.path.exists(VIDEO_JSON):
        print(f"❌ Missing {VIDEO_JSON}")
        return False

    with open(VIDEO_JSON, 'r') as f:
        workflow = json.load(f)

    # Inject Path
    node_found = False
    for node in workflow.values():
        if node.get('class_type') == 'VHS_LoadVideo':
            node['inputs']['video'] = file_path
            node['inputs']['frame_load_cap'] = 0
            if 'upload' in node['inputs']: del node['inputs']['upload']
            node_found = True
            break

    if not node_found:
        print("❌ Error: 'VHS_LoadVideo' node not found in video JSON.")
        return False

    return workflow

def handle_image(file_path, filename):
    print("🖼️ Detected IMAGE. Loading Image Workflow...")
    if not os.path.exists(IMAGE_JSON):
        print(f"❌ Missing {IMAGE_JSON}")
        return False

    # Move to local Comfy input (API Requirement for images)
    temp_path = os.path.join(COMFY_INPUT, filename)
    shutil.copy(file_path, temp_path)

    with open(IMAGE_JSON, 'r') as f:
        workflow = json.load(f)

    # Inject Filename
    node_found = False
    for node in workflow.values():
        if node.get('class_type') == 'LoadImage':
            node['inputs']['image'] = filename
            if 'upload' in node['inputs']: del node['inputs']['upload']
            node_found = True
            break

    if not node_found:
        print("❌ Error: 'LoadImage' node not found in image JSON.")
        return False

    return workflow

# --- MAIN LOOP ---
print(f"🟢 UNIVERSAL WATCHER ACTIVE: {WATCH_FOLDER}")
print("Drop Videos (.mp4) or Images (.png, .jpg) here...")

while True:
    files = sorted(os.listdir(WATCH_FOLDER))
    # Filter valid files
    valid_files = [f for f in files if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv', '.png', '.jpg', '.jpeg', '.webp'))]

    if valid_files:
        target = valid_files[0]
        full_path = os.path.join(WATCH_FOLDER, target)

        print(f"\n🚀 Processing: {target}")

        # Decide workflow based on extension
        workflow = None
        is_video = target.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))

        if is_video:
            workflow = handle_video(full_path, target)
        else:
            workflow = handle_image(full_path, target)

        if workflow:
            try:
                # Send to Engine
                response = queue_prompt(workflow)
                prompt_id = response['prompt_id']
                print(f"--> Job {prompt_id} Queued. Working...")

                # Wait for finish
                start_t = time.time()
                while True:
                    history = get_history(prompt_id)
                    if prompt_id in history:
                        break
                    # 1 hour timeout for video, 2 min for image
                    timeout = 3600 if is_video else 120
                    if time.time() - start_t > timeout:
                        print("❌ Timed out.")
                        break
                    time.sleep(2)

                # Retrieve Output
                print("--> Finished. Moving output...")
                # Get list of files in output dir
                out_files = [os.path.join(COMFY_OUTPUT, f) for f in os.listdir(COMFY_OUTPUT)]

                # Filter for recent files (created in last 5 mins)
                recent = [f for f in out_files if (time.time() - os.path.getmtime(f)) < 300]

                if recent:
                    newest = max(recent, key=os.path.getctime)
                    dest = os.path.join(OUTPUT_FOLDER, f"Upscaled_{target}")
                    shutil.move(newest, dest)
                    print(f"✅ Saved to: {dest}")

                    # Move Original
                    shutil.move(full_path, os.path.join(PROCESSED_FOLDER, target))

                    # Cleanup Temp Image if needed
                    if not is_video:
                        temp_img = os.path.join(COMFY_INPUT, target)
                        if os.path.exists(temp_img): os.remove(temp_img)
                else:
                    print("⚠️ Job finished but output file not found.")

            except Exception as e:
                print(f"❌ Error during processing: {e}")
                time.sleep(5)
        else:
            # Workflow failed to load
            print("❌ Skipping file due to workflow error.")
            time.sleep(5)

    else:
        time.sleep(3)

In [ ]:
# --- ALTERNATIVE: USE NGROK (STABLE) ---
import os
import time
import subprocess
from google.colab import userdata

# 1. Install pyngrok
print("📦 Installing ngrok...")
!pip install pyngrok

from pyngrok import ngrok, conf

# 2. Kill old tunnels to be safe
print("🔪 Killing old tunnels...")
ngrok.kill()
!pkill cloudflared

# 3. Setup Token
# We ask for the token directly to keep it simple
print("\n" + "="*60)
token = input("👉 PASTE YOUR NGROK AUTHTOKEN HERE: ").strip()
print("="*60 + "\n")

if not token:
    print("❌ Error: You must paste a token.")
else:
    # 4. Authenticate
    conf.get_default().auth_token = token

    # 5. Check if ComfyUI is running
    print("🔍 Checking ComfyUI status...")
    # Quick check if something is listening on 8188
    result = os.system("netstat -nlpt | grep :8188 > /dev/null")
    if result != 0:
        print("⚠️ ComfyUI is NOT running. Restarting it...")
        %cd /content/ComfyUI
        if os.path.exists("comfyui.log"): os.remove("comfyui.log")
        !nohup python main.py --listen --port 8188 > comfyui.log 2>&1 &
        print("⏳ Waiting 15s for boot...")
        time.sleep(15)
    else:
        print("✅ ComfyUI is running.")

    # 6. Start Tunnel
    print("🔗 Starting ngrok tunnel...")
    try:
        # Connect to port 8188
        public_url = ngrok.connect(8188).public_url
        print("\n" + "="*60)
        print(f"✅ YOUR STABLE LINK:\n{public_url}")
        print("="*60 + "\n")

        # Keep alive
        print("ℹ️  Tunnel is active. Do not close this cell.")
        # We start a loop to keep the cell running if you want
        # But usually you can just move to the next cell.
    except Exception as e:
        print(f"❌ Error starting ngrok: {e}")